In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
from numpy import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
plan_info = pd.read_csv('./ptom_third_plan.csv')
plan_info.dropna(how='all', inplace=True, axis=1)
plan_info.drop(['ad_start_time', 'ad_end_time', 'remark','origin'], axis=1, inplace=True)
plan_info.dropna(subset=['ad_info'],inplace=True)
plan_info.drop(['ad_id', 'ad_group_id', 'ad_name','media_id','plan_id','launch_op_id','bid_mode','bid_status','bid',
                'budget_mode','budget','opt_status','status','syn_time','update_time','inventory_type'], axis=1, inplace=True)
# 解析json
plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
temp = plan_info['ad_info'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('ad_info', axis=1, inplace=True)
temp = plan_info['audience'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('audience', axis=1, inplace=True)
temp = plan_info['action'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('action', axis=1, inplace=True)
plan_info.dropna(how='all', inplace=True, axis=1)

plan_info_2 = plan_info.copy()
plan_info_2.drop(['game_package_batch_id','advanced_creative_type','game_package_desc','game_package_thumbnail_ids',
               'external_url','intelligent_flow_switch','audience_package_id','union_video_type','converted_time_duration',
               'roi_goal','device_type','deep_bid_type','auto_extend_targets'],axis=1,inplace=True)
plan_info_2.drop(['package','open_url','ad_modify_time','deep_cpabid','download_url','download_mode','id','campaign_id',
               'package','ad_create_time','start_time','ad_id','name','status','modify_time','advertiser_id','schedule_type','quick_app_url',
               'end_time','convert_id','audit_reject_reason','download_type','budget_mode','bid','pricing',
               'include_custom_actions','app_type','exclude_flow_package','aweme_fan_behaviors','business_ids','user_type',
               'activate_type','aweme_fan_accounts','aweme_fan_categories','aweme_fans_numbers','article_category',
                'flow_package','carrier','superior_popularity_type','device_brand','exclude_custom_actions','geolocation',
               'opt_status','learning_phase','district'],axis=1,inplace=True)
# 删除老版兴趣行为定向
plan_info_2.drop(['ad_tag','interest_tags','app_behavior_target','app_category','app_ids'],axis=1,inplace=True)
# 删除老版兴趣行为定向
plan_info_2.drop(['retargeting_tags'],axis=1,inplace=True)

In [4]:
image_info = pd.read_csv('./image_info.csv')
launch_report = pd.read_csv('./launch_report.csv',usecols=['tdate','channel_id', 'source_id', 'amount',
       'create_role_num', 'pay_role_user_num', 'new_role_money'])
df = pd.merge(plan_info_2,image_info,on=['channel_id','source_id'],how='left')
df.dropna(subset=['image_id'],inplace=True)
df.rename(columns={'create_time_x':'create_time','create_time_y':'create_date'},inplace=True)
df = df[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'create_time','create_date',
         'bid2','budget','inventory_type', 'schedule_time', 'delivery_range','flow_control_mode',
        'smart_bid_type', 'adjust_cpa', 'cpa_bid', 'hide_if_exists', 'hide_if_converted',
        'auto_extend_enabled', 'age','gender', 'city', 'location_type', 'platform', 'launch_price','retargeting_type',
         'retargeting_tags_include','retargeting_tags_exclude', 'ac', 'android_osv','ios_osv', 
        'interest_action_mode', 'interest_categories', 'interest_words',
       'action_categories', 'action_days', 'action_scene', 'action_words',
       'image_id', 'label_ids']]
launch_report.sort_values(by='tdate',inplace=True)
launch_report.drop_duplicates(subset=['channel_id','source_id'],keep='first',inplace=True)
df = pd.merge(df, launch_report,on=['channel_id','source_id'],how='left')
df.drop(df[df['tdate'].isna()].index,inplace=True)
df['label'] = ((df['pay_role_user_num']>0)&(df['amount']>=500)).astype(int)
image_score = pd.read_csv('./image_score.csv')
image_score.drop(['image_name','data_win','media_id','label_ids'],axis=1,inplace=True)
df['image_id'] = df['image_id'].astype('int')
image_score.rename(columns={'model_run_datetime':'create_date'},inplace=True)
df = pd.merge(df, image_score, on=['image_id','create_date'],how='left', validate='many_to_one')
df['create_time'] = pd.to_datetime(df['create_time'])

In [7]:
df.drop(['interest_words','action_words'],axis=1,inplace=True)

In [8]:
df.head()

,ad_account_id,game_id,channel_id,source_id,create_time,create_date,bid2,budget,inventory_type,schedule_time,delivery_range,flow_control_mode,smart_bid_type,adjust_cpa,cpa_bid,hide_if_exists,hide_if_converted,auto_extend_enabled,age,gender,city,location_type,platform,launch_price,retargeting_type,retargeting_tags_include,retargeting_tags_exclude,ac,android_osv,ios_osv,interest_action_mode,interest_categories,action_categories,action_days,action_scene,image_id,label_ids,tdate,amount,create_role_num,pay_role_user_num,new_role_money,label,image_run_date_amount,image_create_role_pay_num,image_create_role_num,image_create_role_pay_sum,image_source_num,image_create_role_pay_rate,image_create_role_cost,image_create_role_pay_cost,image_valid_source_num,image_valid_source_rate,image_pay_sum_ability,image_pay_num_ability,image_create_role_roi,image_create_role_retain_1d,score,image_launch_time,image_source_total_num
0,4003,1000840,7162,259913,2020-01-19 16:18:31,2020-01-19,1135.73,5000.0,[INVENTORY_UNION_SLOT],1111111111111111111111111111111111111111111111...,UNION,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,1135.73,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19788,NaN,2020-1-19 00:00:00,30.88,0.0,0.0,0.0,0,15718.76,5.0,293.0,218.0,7.0,0.0171,53.647645,3143.752000,5.0,0.7143,43.600000,1.0000,0.013869,0.1416,621.0,2020-01-14,42.0
1,4003,1000840,7162,259914,2020-01-19 16:18:31,2020-01-19,1135.08,5000.0,"[UNION_BOUTIQUE_GAME, INVENTORY_UNION_SLOT]",1111111111111111111111111111111111111111111111...,DEFAULT,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,1135.08,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19783,NaN,2020-1-19 00:00:00,23.92,5.0,0.0,0.0,0,1372.10,0.0,3.0,0.0,3.0,0.0000,457.366667,inf,0.0,0.0000,0.000000,0.0000,0.000000,0.2500,507.0,2020-01-14,5.0
2,4003,1000840,7162,259915,2020-01-19 16:24:53,2020-01-19,1128.21,5000.0,"[UNION_BOUTIQUE_GAME, INVENTORY_UNION_SLOT]",1111111111111111111111111111111111111111111111...,DEFAULT,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,1128.21,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19636,19,2020-1-19 00:00:00,18.36,2.0,0.0,0.0,0,835691.29,326.0,29232.0,35680.0,102.0,0.0112,28.588235,2563.470215,102.0,1.0000,349.803922,3.1961,0.042695,0.1355,763.0,2019-11-15,854.0
3,4003,1000840,7162,259916,2020-01-20 14:35:23,2020-01-19,1180.24,8000.0,[INVENTORY_UNION_SLOT],1111111111111111111111111111111111111111111111...,UNION,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,1180.24,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19791,NaN,2020-1-20 00:00:00,1599.31,26.0,0.0,0.0,0,1620.13,1.0,42.0,30.0,2.0,0.0238,38.574524,1620.130000,2.0,1.0000,15.000000,0.5000,0.018517,0.0476,623.0,2020-01-14,12.0
4,4003,1000840,7162,259918,2020-01-29 18:01:59,2020-01-19,1377.89,10000.0,"[INVENTORY_FEED, INVENTORY_HOTSOON_FEED, INVEN...",1110000000000000000011111111111111111111111111...,DEFAULT,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,1377.89,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,"[11, 12, 13, 14, 15, 21, 22, 23, 31, 32, 33, 3...",CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19636,19,2020-1-29 00:00:00,6997.97,300.0,4.0,268.0,1,835691.29,326.0,29232.0,35680.0,102.0,0.0112,28.588235,2563.470215,102.0,1.0000,349.803922,3.1961,0.042695,0.1355,763.0,2019-11-15,854.0


In [198]:
# 时间序列数据
# seq_df = df.drop(['create_date','channel_id','source_id','create_date','tdate','label','image_launch_time'],axis=1)

In [205]:
seq_df = df[['ad_account_id','game_id','create_time','platform','image_id', 'label_ids','score','amount',
       'create_role_num', 'pay_role_user_num', 'new_role_money']]

In [206]:
seq_df.head()

,ad_account_id,game_id,create_time,platform,image_id,label_ids,score,amount,create_role_num,pay_role_user_num,new_role_money
0,4003,1000840,2020-01-19 16:18:31,[ANDROID],19788,NaN,621.0,30.88,0.0,0.0,0.0
1,4003,1000840,2020-01-19 16:18:31,[ANDROID],19783,NaN,507.0,23.92,5.0,0.0,0.0
2,4003,1000840,2020-01-19 16:24:53,[ANDROID],19636,19,763.0,18.36,2.0,0.0,0.0
3,4003,1000840,2020-01-20 14:35:23,[ANDROID],19791,NaN,623.0,1599.31,26.0,0.0,0.0
4,4003,1000840,2020-01-29 18:01:59,[ANDROID],19636,19,763.0,6997.97,300.0,4.0,268.0


In [200]:
# seq_df['image_create_role_cost'] = pd.cut(seq_df.image_create_role_cost, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
# seq_df['image_create_role_pay_cost'] = pd.cut(seq_df.image_create_role_pay_cost, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])

In [181]:
# cols_num = ['cpa_bid','image_pay_num_ability','image_pay_sum_ability','pay_role_user_num','image_valid_source_rate','image_create_role_cost',
#  'image_create_role_pay_cost','image_create_role_roi','bid2','new_role_money','score','image_create_role_pay_rate','image_run_date_amount','image_source_total_num',
#  'image_create_role_num','amount','image_create_role_pay_sum','adjust_cpa','hide_if_exists','budget','image_source_num','image_valid_source_num',
#  'image_create_role_pay_num','auto_extend_enabled','create_role_num','image_create_role_retain_1d']

In [166]:
# for col in cols_buckets:
#     seq_df[col] = seq_df[col].fillna(0)
#     seq_df[col] = seq_df[col].astype(float)

In [168]:
# seq_df['cpa_bid'] = pd.cut(seq_df.cpa_bid, [float('-inf'), 1000,2000,3000,4000, float('inf')], labels=[1,2,3,4,5])
# seq_df['image_pay_num_ability'] = pd.cut(seq_df.image_pay_num_ability, [float('-inf'), 0,2,4, float('inf')], labels=[1,2,3,4])
# seq_df['image_pay_sum_ability'] = pd.cut(seq_df.image_pay_sum_ability, [float('-inf'), 0,200,300,500, float('inf')], labels=[1,2,3,4,5])
# seq_df['pay_role_user_num'] = pd.cut(seq_df.pay_role_user_num, [float('-inf'), 0,1,2,3, float('inf')], labels=[1,2,3,4,5])


In [197]:
# sns.distplot(seq_df['image_create_role_cost'])
# plt.show()

In [124]:
cols_mutil = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']

In [125]:
cols_one = ['ad_account_id','game_id','delivery_range','flow_control_mode','smart_bid_type','hide_if_converted','gender',
           'location_type','platform','launch_price','retargeting_type','android_osv','ios_osv','interest_action_mode',
           'action_days','image_id','label_ids','image_create_role_cost','image_create_role_pay_cost']

In [95]:
seq_cat_cols = ['ad_account_id','game_id','delivery_range','flow_control_mode','smart_bid_type','hide_if_converted','gender',
           'location_type','platform','launch_price','retargeting_type','android_osv','ios_osv','interest_action_mode',
           'action_days','image_id','label_ids']
for col in seq_cat_cols:
    seq_df[col] = seq_df[col].astype(str)

In [201]:
def get_featrue(data):

    cols_mutil = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']
    for col in cols_mutil:
        if col in ['inventory_type','age']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
        else:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
            
    cols_one = ['ad_account_id','game_id','delivery_range','flow_control_mode','smart_bid_type','hide_if_converted','gender',
           'location_type','platform','launch_price','retargeting_type','android_osv','ios_osv','interest_action_mode',
           'action_days','image_id','label_ids','image_create_role_cost','image_create_role_pay_cost']
    for col in cols_one:
        data[col] = data[col].astype(str)
        data = data.join(pd.get_dummies(data[col],prefix=col))
        data.drop(col,axis=1,inplace=True)
        
    data['schedule_time'] = data['schedule_time'].astype(str)
    temp = data['schedule_time'].str.join('|').str.split('|',expand=True)
    data = data.join(temp)
    del temp
    data.drop('schedule_time',axis=1,inplace=True)
    
    # 数值列
    for col in list(set(data.columns.to_list()) - set(cols_mutil+cols_one+['create_time']+['schedule_time'])):
        data[col] = data[col].fillna(0)
        data[col] = data[col].astype(np.float32)
        mm = MinMaxScaler()
        data[col] = mm.fit_transform(data[col].values.reshape(-1,1))
    
    gc.collect()
    return data

In [203]:
%%time
# seq_df = get_featrue(seq_df)

Wall time: 0 ns


In [99]:
seq_df.columns

Index(['ad_account_id', 'game_id', 'create_time', 'bid2', 'budget',
       'inventory_type', 'schedule_time', 'delivery_range',
       'flow_control_mode', 'smart_bid_type', 'adjust_cpa', 'cpa_bid',
       'hide_if_exists', 'hide_if_converted', 'auto_extend_enabled', 'age',
       'gender', 'city', 'location_type', 'platform', 'launch_price',
       'retargeting_type', 'retargeting_tags_include',
       'retargeting_tags_exclude', 'ac', 'android_osv', 'ios_osv',
       'interest_action_mode', 'interest_categories', 'action_categories',
       'action_days', 'action_scene', 'image_id', 'label_ids', 'amount',
       'create_role_num', 'pay_role_user_num', 'new_role_money',
       'image_run_date_amount', 'image_create_role_pay_num',
       'image_create_role_num', 'image_create_role_pay_sum',
       'image_source_num', 'image_create_role_pay_rate',
       'image_create_role_cost', 'image_create_role_pay_cost',
       'image_valid_source_num', 'image_valid_source_rate',
       'image_pay_

In [57]:
# # 对列表内容进行编码降维
def get_featrue(data):
    cols = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']
    for col in cols:
        if col in ['inventory_type','age']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
        elif col in ['city','retargeting_tags_include','retargeting_tags_exclude','interest_categories','action_categories']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            temp = data[col].str.join('|').str.get_dummies()
    #         print(temp.shape[1])
            pca = PCA(n_components=0.9)
            temp = pca.fit_transform(temp.values)
            temp = pd.DataFrame(temp,columns = [col+str(i) for i in range(temp.shape[1])])
    #         print(temp.shape[1])
            data = data.join(temp)
            del temp
            data.drop(col,axis=1,inplace=True)
        else:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
    data['schedule_time'] = data['schedule_time'].astype(str)
    temp = data['schedule_time'].str.join('|').str.split('|',expand=True)
    pca = PCA(n_components=0.9)
    temp = pca.fit_transform(temp.values)
    temp = pd.DataFrame(temp,columns = ['schedule_time'+str(i) for i in range(temp.shape[1])])
    data = data.join(temp)
    del temp
    data.drop('schedule_time',axis=1,inplace=True)
    gc.collect()
    return data

In [21]:
# 对schedule_time进行编码
# def get_schedule_time(data):
#     data['schedule_time'] = data['schedule_time'].astype(str)
#     temp = data['schedule_time'].str.join('|').str.split('|',expand=True)
#     pca = PCA(n_components=0.9)
#     temp = pca.fit_transform(temp.values)
#     temp = pd.DataFrame(temp,columns = ['schedule_time'+str(i) for i in range(temp.shape[1])])
#     data = data.join(temp)
#     del temp
#     data.drop('schedule_time',axis=1,inplace=True)
#     return data

In [117]:
seq_df.shape

(28562, 4743)

In [118]:
seq_df.head()

create_time      bid2   budget  adjust_cpa   cpa_bid  \
0 2020-01-19 16:18:31  0.162143  0.00047         0.0  0.162143   
1 2020-01-19 16:18:31  0.162143  0.00047         0.0  0.162143   
2 2020-01-19 16:24:53  0.161143  0.00047         0.0  0.161143   
3 2020-01-20 14:35:23  0.168571  0.00077         0.0  0.168571   
4 2020-01-29 18:01:59  0.196714  0.00097         0.0  0.196714   

   hide_if_exists  auto_extend_enabled    amount  create_role_num  \
0             0.0                  0.0  0.000304         0.000000   
1             0.0                  0.0  0.000233         0.001080   
2             0.0                  0.0  0.000182         0.000432   
3             0.0                  0.0  0.016186         0.005618   
4             0.0                  0.0  0.070829         0.064823   

   pay_role_user_num  new_role_money  image_run_date_amount  \
0           0.000000        0.000000               0.010262   
1           0.000000        0.000000               0.000896   
2           0.000000        0.000000               0.545590   
3           0.000000        0.000000               0.001058   
4           0.148148        0.055303               0.545590   

   image_create_role_pay_num  image_create_role_num  \
0                   0.010373               0.003610   
1                   0.000000               0.000037   
2                   0.676349               0.360195   
3                   0.002075               0.000518   
4                   0.676349               0.360195   

   image_create_role_pay_sum  image_source_num  image_create_role_pay_rate  \
0                   0.003742          0.031532                         0.0   
1                   0.000000          0.013514                         0.0   
2                   0.612448          0.459459                         0.0   
3                   0.000515          0.009009                         0.0   
4                   0.612448          0.459459                         0.0   

   image_create_role_cost  image_create_role_pay_cost  image_valid_source_num  \
0                0.006585                    0.064323                0.028409   
1                0.056777                    0.000000                0.000000   
2                0.003479                    0.052453                0.579545   
3                0.004721                    0.033154                0.011364   
4                0.003479                    0.052453                0.579545   

   image_valid_source_rate  image_pay_sum_ability  image_pay_num_ability  \
0                      0.0               0.003825               0.012195   
1                      0.0               0.000000               0.000000   
2                      1.0               0.031044               0.036585   
3                      1.0               0.001334               0.000000   
4                      1.0               0.031044               0.036585   

   image_create_role_roi  image_create_role_retain_1d     score  \
0                    0.0                          0.0  0.759169   
1                    0.0                          0.0  0.619804   
2                    0.0                          0.0  0.932763   
3                    0.0                          0.0  0.761614   
4                    0.0                          0.0  0.932763   

   image_source_total_num  inventory_type_INVENTORY_AWEME_FEED  \
0                0.049180                                  0.0   
1                0.005855                                  0.0   
2                1.000000                                  0.0   
3                0.014052                                  0.0   
4                1.000000                                  1.0   

   inventory_type_INVENTORY_AWEME_SEARCH  inventory_type_INVENTORY_FEED  \
0                                    0.0                            0.0   
1                                    0.0                            0.0   
2                                    0.0          

In [66]:
# seq_cat_cols = ['ad_account_id','game_id','delivery_range','flow_control_mode','smart_bid_type','hide_if_converted','gender',
#            'location_type','platform','launch_price','retargeting_type','android_osv','ios_osv','interest_action_mode',
#            'action_days','image_id','label_ids']
# for col in seq_cat_cols:
#     seq_df[col] = seq_df[col].astype(str)
#     LE = LabelEncoder()
#     seq_df[col] = LE.fit_transform(seq_df[col])

In [120]:
seq_df.loc[0:5,'image_create_role_pay_rate']

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: image_create_role_pay_rate, dtype: float64

In [67]:
seq_df.head()

,ad_account_id,game_id,create_time,bid2,budget,delivery_range,flow_control_mode,smart_bid_type,adjust_cpa,cpa_bid,hide_if_exists,hide_if_converted,auto_extend_enabled,gender,location_type,platform,launch_price,retargeting_type,android_osv,ios_osv,interest_action_mode,action_days,image_id,label_ids,amount,create_role_num,pay_role_user_num,new_role_money,image_run_date_amount,image_create_role_pay_num,image_create_role_num,image_create_role_pay_sum,image_source_num,image_create_role_pay_rate,image_create_role_cost,image_create_role_pay_cost,image_valid_source_num,image_valid_source_rate,image_pay_sum_ability,image_pay_num_ability,image_create_role_roi,image_create_role_retain_1d,score,image_source_total_num,inventory_type_INVENTORY_AWEME_FEED,inventory_type_INVENTORY_AWEME_SEARCH,inventory_type_INVENTORY_FEED,inventory_type_INVENTORY_HOTSOON_FEED,inventory_type_INVENTORY_SEARCH,inventory_type_INVENTORY_UNION_SLOT,inventory_type_INVENTORY_UNION_SPLASH_SLOT,inventory_type_INVENTORY_VIDEO_FEED,inventory_type_UNION_BOUTIQUE_GAME,age_AGE_ABOVE_50,age_AGE_BETWEEN_18_23,age_AGE_BETWEEN_24_30,age_AGE_BETWEEN_31_40,age_AGE_BETWEEN_41_49,city0,city1,city2,retargeting_tags_include0,retargeting_tags_include1,retargeting_tags_include2,retargeting_tags_include3,retargeting_tags_include4,retargeting_tags_include5,retargeting_tags_include6,retargeting_tags_include7,retargeting_tags_include8,retargeting_tags_include9,retargeting_tags_include10,retargeting_tags_include11,retargeting_tags_include12,retargeting_tags_include13,retargeting_tags_include14,retargeting_tags_include15,retargeting_tags_include16,retargeting_tags_include17,retargeting_tags_include18,retargeting_tags_include19,retargeting_tags_include20,retargeting_tags_include21,retargeting_tags_include22,retargeting_tags_include23,retargeting_tags_include24,retargeting_tags_include25,retargeting_tags_include26,retargeting_tags_include27,retargeting_tags_include28,retargeting_tags_include29,retargeting_tags_include30,retargeting_tags_include31,retargeting_tags_include32,retargeting_tags_include33,retargeting_tags_include34,retargeting_tags_include35,retargeting_tags_include36,retargeting_tags_include37,retargeting_tags_include38,retargeting_tags_include39,retargeting_tags_include40,retargeting_tags_include41,retargeting_tags_include42,retargeting_tags_include43,retargeting_tags_include44,retargeting_tags_include45,retargeting_tags_include46,retargeting_tags_include47,retargeting_tags_include48,retargeting_tags_include49,retargeting_tags_include50,retargeting_tags_include51,retargeting_tags_include52,retargeting_tags_include53,retargeting_tags_include54,retargeting_tags_include55,retargeting_tags_include56,retargeting_tags_include57,retargeting_tags_include58,retargeting_tags_include59,retargeting_tags_include60,retargeting_tags_include61,retargeting_tags_include62,retargeting_tags_include63,retargeting_tags_include64,retargeting_tags_include65,retargeting_tags_include66,retargeting_tags_include67,retargeting_tags_include68,retargeting_tags_include69,retargeting_tags_include70,retargeting_tags_include71,retargeting_tags_include72,retargeting_tags_include73,retargeting_tags_include74,retargeting_tags_include75,retargeting_tags_include76,retargeting_tags_include77,retargeting_tags_include78,retargeting_tags_include79,retargeting_tags_include80,retargeting_tags_include81,retargeting_tags_include82,retargeting_tags_include83,retargeting_tags_include84,retargeting_tags_include85,retargeting_tags_include86,retargeting_tags_include87,retargeting_tags_include88,retargeting_tags_include89,retargeting_tags_include90,retargeting_tags_include91,retargeting_tags_include92,retargeting_tags_include93,retargeting_tags_include94,retargeting_tags_include95,retargeting_tags_include96,retargeting_tags_include97,retargeting_tags_include98,retargeting_tags_include99,retargeting_tags_include100,retargeting_tags_include101,retargeting_tags_include102,retargeting_tags_include103,retargeting_tags_include104,retarget

In [55]:
def get_feature_columns(data):
    feature_columns = []
    feature_layer_inputs = {}
    seq_cat_cols = ['ad_account_id','game_id','delivery_range','flow_control_mode','smart_bid_type','hide_if_converted','gender',
           'location_type','platform','launch_price','retargeting_type','android_osv','ios_osv','interest_action_mode',
           'action_days','image_id','label_ids']
    for col in seq_cat_cols:
        data[col] = data[col].astype(str)
    # 数值列
    for col in list(set(data.columns.to_list()) - set(seq_cat_cols+['label'])):
        feature_columns.append(tf.feature_column.numeric_column(col))
        feature_layer_inputs[col] = tf.keras.Input(shape=(1,), name=col)

    # 分桶列
    # budget = tf.feature_column.numeric_column('budget')
    # budget_buckets = tf.feature_column.bucketized_column(budget, 
    #              boundaries=[5000,7000, 10000])
    # feature_columns.append(budget_buckets)

    # 类别列
    # 注意：所有的Catogorical Column类型最终都要通过indicator_column转换成Dense Column类型才能传入模型！！
    # for col in cat_cols:
    #     temp = tf.feature_column.indicator_column(
    #           tf.feature_column.categorical_column_with_vocabulary_list(key=col,data[col].unique().tolist()))
    #     feature_columns.append(temp)

    for col in seq_cat_cols:
        temp = tf.feature_column.categorical_column_with_vocabulary_list(col,data[col].unique().tolist())
        if (data[col].nunique()>10) & (data[col].nunique()<100):
            temp = tf.feature_column.embedding_column(temp,dimension=8)
        elif (data[col].nunique()>100) & (data[col].nunique()<1000):
            temp = tf.feature_column.embedding_column(temp,dimension=15)
        elif data[col].nunique()>1000:
            temp = tf.feature_column.embedding_column(temp,dimension=22)
        else: 
            temp = tf.feature_column.indicator_column(temp)
        feature_columns.append(temp)
        feature_layer_inputs[col] = tf.keras.Input(shape=(1,), name=col, dtype=tf.string)
    
    return feature_columns,feature_layer_inputs


In [60]:
feature_columns,feature_layer_inputs = get_feature_columns(seq_df)